In [1]:
%cd ..
%load_ext autoreload
%autoreload 2

/mnt/data4/haryoaw_workspace/projects/2021_2/s4_happy/happy_s4


In [37]:
from happy_s4.data.lit_data import HFLitDataArgs, HFLitDataModule
from happy_s4.model.lit_model import LitS4Args, LitS4
from happy_s4.model.s4_wrap import S4_GO_BRR_ARGS
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
from happy_s4.data.data import WordTokenizer, shape_dataset
from happy_s4.data.collators import BatchCollators
import datasets
from functools import partial
from torch.utils.data import DataLoader
import datasets
from functools import partial
from torch.utils.data import DataLoader
import torch
from tqdm import tqdm
from sklearn.metrics import accuracy_score
import munch
from pathlib import Path

In [3]:
from dataclasses import dataclass

In [4]:
"""
Mixin module
"""

class MappingMixin(dict):
    def __setitem__(self, key, item):
        self.__dict__[key] = item

    def __getitem__(self, key):
        return self.__dict__[key]

    def __repr__(self):
        return repr(self.__dict__)

    def __len__(self):
        return len(self.__dict__)

    def __delitem__(self, key):
        del self.__dict__[key]

    def clear(self):
        return self.__dict__.clear()

    def copy(self):
        return self.__dict__.copy()

    def has_key(self, k):
        return k in self.__dict__

    def update(self, *args, **kwargs):
        return self.__dict__.update(*args, **kwargs)

    def keys(self):
        return self.__dict__.keys()

    def values(self):
        return self.__dict__.values()

    def items(self):
        return self.__dict__.items()

    def pop(self, *args):
        return self.__dict__.pop(*args)

    def __cmp__(self, dict_):
        return self.__cmp__(self.__dict__, dict_)

    def __contains__(self, item):
        return item in self.__dict__

    def __iter__(self):
        return iter(self.__dict__)

In [24]:
data_inp_list = {
    "superglue/rte": {
        "data_name": "super_glue",
        "data_subset": "rte",
        "column_train": ["premise", "hypothesis"],
        "column_label": "label"
    },
    "glue/mrpc": {
        "data_name": "glue",
        "data_subset": "mrpc",
        "column_train": ["sentence1", "sentence2"],
        "column_label": "label"
    },
    "glue/sst2": {
        "data_name": "glue",
        "data_subset": "sst2",
        "column_train": ["sentence"],
        "column_label": "label"
    },
    "glue/qqp": {
        "data_name": "glue",
        "data_subset": "qqp",
        "column_train": ["question1", "question2"],
        "column_label": "label"
    },
    "glue/qnli": {
        "data_name": "glue",
        "data_subset": "qnli",
        "column_train": ["question", "sentence"],
        "column_label": "label"
    },
    "glue/rte": {
        "data_name": "glue",
        "data_subset": "rte",
        "column_train": ["sentence1", "sentence2"],
        "column_label": "label"
    },
    "glue/wnli": {
        "data_name": "glue",
        "data_subset": "wnli",
        "column_train": ["sentence1", "sentence2"],
        "column_label": "label"
    },
}

In [25]:
@dataclass
class BatchArgs(MappingMixin):
    """
    Batch Size Args. The formula is:
    ```
    batch_size = total_batch_size / grad_accumulation / num_gpus (args in Trainer)
    ```
    """
    total_batch_size: int = 64
    grad_accumulation: int = 1

In [26]:
trainer_args = {
    "gpus": [0],
    "max_epochs": 1000,
}

In [27]:
model_args = {
    "d_model": 128,
    "l_max": 4000,
    "channels": 1,
    "hurwitz": True,
    "transposed": False,
    "dropout": 0.0,
    "n_layers": 4,
    "trainable": {"dt": True, "A": True, "P": True, "B": True},
    "bidirectional": True
}

In [28]:
# SEE HFLITDATAARGS
batch_args = {
    "total_batch_size": 64,
    "grad_accumulation": 1,    
}
data_args = {
    "data": "superglue/rte",
    "args": {
        "split_pattern": "",
    }
}

In [29]:
early_stopping_args = dict(
    monitor="val_loss",
    patience=5
)
model_checkpoint_args = dict(
    filename="checkpoint_{epoch:02d}-{val_loss:.4f}", 
    monitor='val_loss'
)

In [30]:
model_name = 's4-exp-1'

In [31]:
from pytorch_lightning import seed_everything

In [32]:
seed_everything(1234, workers=True)

Global seed set to 1234


1234

In [33]:
model_args

{'d_model': 128,
 'l_max': 4000,
 'channels': 1,
 'hurwitz': True,
 'transposed': False,
 'dropout': 0.0,
 'n_layers': 4,
 'trainable': {'dt': True, 'A': True, 'P': True, 'B': True},
 'bidirectional': True}

In [34]:
def run_train(out_dir, data_args, batch_args, trainer_args, 
              model_args, early_stopping_args, model_ckeckpoint_args):
    batch_args = BatchArgs(**batch_args)
    batch_size = int(
        batch_args.total_batch_size / batch_args.grad_accumulation / len(trainer_args['gpus'])
    )
    lit_data_args = data_args.get('args')
    lit_data_args.update(data_inp_list.get(data_args.get("data"), {}))
    lit_data_args['batch_size'] = batch_size
    lit_data_args = HFLitDataArgs(**lit_data_args)
    dm = HFLitDataModule(lit_data_args)
    dm.setup()
    vocab_size = len(dm.tokenizer.vocab.get_itos())
    pad_token_id = dm.tokenizer.pad_token_id
    s4_args = S4_GO_BRR_ARGS(pad_token_id=pad_token_id, vocab_size=vocab_size, **model_args)
    lit_s4_args = LitS4Args(model_args=s4_args)
    lit_s4 = LitS4(lit_s4_args)
    es = EarlyStopping(**early_stopping_args)
    mckpt = ModelCheckpoint(dirpath=out_dir, **model_checkpoint_args)
    trainer = Trainer(callbacks=[es, mckpt], accumulate_grad_batches=batch_args.grad_accumulation, **trainer_args)
    trainer.fit(model=lit_s4, datamodule=dm)
    with open(Path(out_dir) / "best_model_path.txt", "w+") as file:
        file.write(mckpt.best_model_path)
        file.write(f" {mckpt.best_model_score.item()}")

In [ ]:
for data in data_inp_list:
    print(f"{data} train")
    out_dir = f"outputs/{model_name}/{data}"
    data_args['data'] = data
    run_train(out_dir, data_args, batch_args, trainer_args, model_args, early_stopping_args,
              model_checkpoint_args)
    

glue/qqp train


Reusing dataset glue (/mnt/data1/hf_dataset_cache/glue/qqp/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
100%|█████████████████████████████████████████████████████████████████████████| 363846/363846 [01:39<00:00, 3664.29ex/s]
Reusing dataset glue (/mnt/data1/hf_dataset_cache/glue/qqp/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
100%|███████████████████████████████████████████████████████████████████████████| 40430/40430 [00:11<00:00, 3565.25ex/s]


{'dt': True, 'A': True, 'P': True, 'B': True}
{'dt': True, 'A': True, 'P': True, 'B': True}
{'dt': True, 'A': True, 'P': True, 'B': True}


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


{'dt': True, 'A': True, 'P': True, 'B': True}


Reusing dataset glue (/mnt/data1/hf_dataset_cache/glue/qqp/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
100%|█████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 20.79it/s]
Reusing dataset glue (/mnt/data1/hf_dataset_cache/glue/qqp/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
Loading cached processed dataset at /mnt/data1/hf_dataset_cache/glue/qqp/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-05c18db07bcad431.arrow
Reusing dataset glue (/mnt/data1/hf_dataset_cache/glue/qqp/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
Loading cached processed dataset at /mnt/data1/hf_dataset_cache/glue/qqp/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-4a7759f1c657b5b2.arrow
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name  | Type                     | Params
-------------------------------------------------

Epoch 0:  90%|███████████████████████████████████████████▏    | 5686/6318 [05:28<00:36, 17.32it/s, loss=0.546, v_num=18]
Validation: 0it [00:00, ?it/s]
Epoch 1:  90%|███████████████████████████████████████████▏    | 5686/6318 [11:26<01:16,  8.28it/s, loss=0.503, v_num=18]
Validation: 0it [00:00, ?it/s]
Epoch 2:  90%|███████████████████████████████████████████▏    | 5686/6318 [17:19<01:55,  5.47it/s, loss=0.472, v_num=18]
Validation: 0it [00:00, ?it/s]
Epoch 3:  35%|████████████████▉                               | 2223/6318 [19:50<36:32,  1.87it/s, loss=0.461, v_num=18]